In [1]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from NYUDv2Loader import *

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
data_path = '/home/dongwonshin/Desktop/Datasets/NYUDv2/'
arg_string = '--arch fcn8s_rgbd_renet_maskedconv --batch_size 1 --model_path fcn8s_rgbd_renet_maskedconv_NYUDv2_best_model.pkl'

# Argument setting

In [2]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.utils import convert_state_dict

from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=100, help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None, help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', help='Disable visualization(s) on visdom | False by default')
parser.add_argument('--model_path', nargs='?', type=str, default='fcn8s_pascal_1_26.pkl', help='Path to the saved model')
parser.set_defaults(visdom=False)

In [3]:
args = parser.parse_args(arg_string.split(' '))

# Setup Augmentations
data_aug= Compose([RandomRotate(10), RandomHorizontallyFlip()])

# Setup Dataloader
t_loader = NYUDv2Loader(data_path, is_transform=True)
v_loader = NYUDv2Loader(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)



In [4]:

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))


In [5]:
# Setup Model
model = get_model('fcn8s_rgbd_renet_maskedconv', n_classes)
state = convert_state_dict(torch.load('fcn8s_rgbd_renet_maskedconv_NYUDv2_best_model.pkl')['model_state'])
model.load_state_dict(state)
model.cuda()

fcn8s_rgbd_renet_maskedconv(
  (conv_block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(100, 100))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  )
  (depth_conv_block1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(100, 100))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  )
  (conv_block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  )
  (conv_

In [6]:

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 

# Retraining

In [ ]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        images = Variable(color_imgs.cuda())
        depth_images = Variable(depth_imgs.cuda())
        labels = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(images,depth_images)

        loss = loss_fn(input=outputs, target=labels)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 20 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val, depth_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "../model_weights/{}_{}_best_model.pkl".format(args.arch, 'NYUDv2'))

/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/100] Loss: 0.6700
Epoch [1/100] Loss: 0.4108
Epoch [1/100] Loss: 0.3484
Epoch [1/100] Loss: 0.5087
Epoch [1/100] Loss: 0.6477
Epoch [1/100] Loss: 0.5684
Epoch [1/100] Loss: 0.4184
Epoch [1/100] Loss: 0.2865
Epoch [1/100] Loss: 0.6011
Epoch [1/100] Loss: 0.5326
Epoch [1/100] Loss: 0.3604
Epoch [1/100] Loss: 0.8140
Epoch [1/100] Loss: 0.3643
Epoch [1/100] Loss: 0.5722
Epoch [1/100] Loss: 0.6673
Epoch [1/100] Loss: 0.4742
Epoch [1/100] Loss: 0.6045
Epoch [1/100] Loss: 0.3847
Epoch [1/100] Loss: 0.3200
Epoch [1/100] Loss: 0.6080
Epoch [1/100] Loss: 0.6261
Epoch [1/100] Loss: 0.4750
Epoch [1/100] Loss: 0.6540
Epoch [1/100] Loss: 0.5345
Epoch [1/100] Loss: 0.4717
Epoch [1/100] Loss: 0.6687
Epoch [1/100] Loss: 0.3037
Epoch [1/100] Loss: 0.2690
Epoch [1/100] Loss: 0.4513
Epoch [1/100] Loss: 0.5540
Epoch [1/100] Loss: 0.4336
Epoch [1/100] Loss: 0.6925
Epoch [1/100] Loss: 0.7310
Epoch [1/100] Loss: 0.5243
Epoch [1/100] Loss: 0.4604
Epoch [1/100] Loss: 0.4704
Epoch [1/100] Loss: 0.3713
E

654it [01:49,  6.00it/s]


Overall Acc: 	 0.563596512447
Mean Acc : 	 0.460810027112
FreqW Acc : 	 0.396077814798
Mean IoU : 	 0.333418897682
Epoch [2/100] Loss: 0.5707
Epoch [2/100] Loss: 0.4628
Epoch [2/100] Loss: 0.9656
Epoch [2/100] Loss: 0.4752
Epoch [2/100] Loss: 0.3990
Epoch [2/100] Loss: 0.8508
Epoch [2/100] Loss: 0.4215
Epoch [2/100] Loss: 0.6424
Epoch [2/100] Loss: 0.6179
Epoch [2/100] Loss: 0.4009
Epoch [2/100] Loss: 0.4119
Epoch [2/100] Loss: 0.3982
Epoch [2/100] Loss: 0.6553
Epoch [2/100] Loss: 0.4372
Epoch [2/100] Loss: 0.3871
Epoch [2/100] Loss: 0.7075
Epoch [2/100] Loss: 0.6786
Epoch [2/100] Loss: 0.4405
Epoch [2/100] Loss: 0.4859
Epoch [2/100] Loss: 0.5640
Epoch [2/100] Loss: 0.4611
Epoch [2/100] Loss: 0.5598
Epoch [2/100] Loss: 0.8004
Epoch [2/100] Loss: 0.4566
Epoch [2/100] Loss: 0.6114
Epoch [2/100] Loss: 0.8653
Epoch [2/100] Loss: 0.5794
Epoch [2/100] Loss: 0.4394
Epoch [2/100] Loss: 0.5943
Epoch [2/100] Loss: 0.7529
Epoch [2/100] Loss: 0.4005
Epoch [2/100] Loss: 0.4531
Epoch [2/100] Loss: 0

654it [01:48,  6.00it/s]


...

Overall Acc: 	 0.586008572049
Mean Acc : 	 0.476342572972
FreqW Acc : 	 0.42598588283
Mean IoU : 	 0.349439349921


Epoch [54/100] Loss: 0.3056
Epoch [54/100] Loss: 0.1488
Epoch [54/100] Loss: 0.3620
Epoch [54/100] Loss: 0.3741
Epoch [54/100] Loss: 0.5106
Epoch [54/100] Loss: 0.2358
Epoch [54/100] Loss: 0.2174
Epoch [54/100] Loss: 0.1686
Epoch [54/100] Loss: 0.2660
Epoch [54/100] Loss: 0.3992
Epoch [54/100] Loss: 0.2570
Epoch [54/100] Loss: 0.3859
Epoch [54/100] Loss: 0.1192
Epoch [54/100] Loss: 0.2191
Epoch [54/100] Loss: 0.3613
Epoch [54/100] Loss: 0.2820
Epoch [54/100] Loss: 0.3974
Epoch [54/100] Loss: 0.4390
Epoch [54/100] Loss: 0.2421
Epoch [54/100] Loss: 0.3140
Epoch [54/100] Loss: 0.2722
Epoch [54/100] Loss: 0.2196
Epoch [54/100] Loss: 0.2761
Epoch [54/100] Loss: 0.3187
Epoch [54/100] Loss: 0.2049
Epoch [54/100] Loss: 0.4216
Epoch [54/100] Loss: 0.2235
Epoch [54/100] Loss: 0.2661
Epoch [54/100] Loss: 0.2513
Epoch [54/100] Loss: 0.2420
Epoch [54/100] Loss: 0.4302
Epoch [54/100] Loss: 0.1699
Epoch [54/100] Loss: 0.2608
Epoch [54/100] Loss: 0.2713
Epoch [54/100] Loss: 0.3305
Epoch [54/100] Loss:

654it [01:48,  6.01it/s]

Overall Acc: 	 0.584782139956
Mean Acc : 	 0.474295932931
FreqW Acc : 	 0.422570846634
Mean IoU : 	 0.35265345897


Epoch [55/100] Loss: 0.2768
Epoch [55/100] Loss: 0.2778
Epoch [55/100] Loss: 0.2841
Epoch [55/100] Loss: 0.2196
Epoch [55/100] Loss: 0.2990
Epoch [55/100] Loss: 0.2873
Epoch [55/100] Loss: 0.3873
Epoch [55/100] Loss: 0.2741
Epoch [55/100] Loss: 0.2085
Epoch [55/100] Loss: 0.2189
Epoch [55/100] Loss: 0.3650
Epoch [55/100] Loss: 0.1988
Epoch [55/100] Loss: 0.3924
Epoch [55/100] Loss: 0.2485
Epoch [55/100] Loss: 0.3046
Epoch [55/100] Loss: 0.3836
Epoch [55/100] Loss: 0.2472
Epoch [55/100] Loss: 0.2081
Epoch [55/100] Loss: 0.2031
Epoch [55/100] Loss: 0.5681
Epoch [55/100] Loss: 0.2961
Epoch [55/100] Loss: 0.2711
Epoch [55/100] Loss: 0.1617
Epoch [55/100] Loss: 0.4090
Epoch [55/100] Loss: 0.1708
Epoch [55/100] Loss: 0.2394
Epoch [55/100] Loss: 0.1533
Epoch [55/100] Loss: 0.4388
Epoch [55/100] Loss: 0.4919
Epoch [55/100] Loss: 0.1928
Epoch [55/100] Loss: 0.2637
Epoch [55/100] Loss: 0.4413
Epoch [55/100] Loss: 0.1752
Epoch [55/100] Loss: 0.4715


# Training.. fcn8s_rgbd_renet_maskedconv on NYUDv2